# Tier 3, Week 5: Multi-Qubit Circuits & Introduction to Noise

**Audience:** Undergraduate / Advanced

**Objective:** Generalize from 2-qubit circuits to complex multi-qubit gates (Toffoli, $C^n(U)$). Introduce the standard circuit description language (OpenQASM) and the formal mathematical tool for modeling noise (the Density Matrix).

--- 

## 1. Bridging from Past Weeks: The Limits of Our Model

* **Week 3 (Tensor Products):** We learned that a 2-qubit system is a $2^2=4$-dimensional vector. A 3-qubit system is $2^3=8$-dimensional. A 10-qubit system is $2^{10}=1024$-dimensional. A 50-qubit system has a state vector with $2^{50} \approx 10^{15}$ amplitudes.

**Thus,** we cannot write down the state vector or matrix for a 50-qubit system. It's computationally impossible. We must stop thinking about *simulating* circuits with state vectors (which is slow) and start thinking about *describing* them with a formal language. This is the move from simulation to true quantum algorithm design.

## 2. Beyond CNOT: Generalizing Controlled Gates

In Week 3, we used the `CNOT` ($CX$) gate, which is a $C^1(X)$ gate (a *singly-controlled* NOT). To build useful algorithms, we need more powerful "if-then" logic.

### Tier 2 Bridge: The Toffoli & SWAP Gates

**1. Toffoli Gate ($CCX$ or $C^2(X)$):**

* It is a *doubly-controlled* NOT gate. It has two control qubits and one target qubit. This means that it flips the target qubit *if and only if* both control qubits are in the $|1\rangle$ state.
* **Example:** $CCX|110\rangle \to |111\rangle$
* **Example:** $CCX|100\rangle \to |100\rangle$ (no change, only one control is 1)

This is a 3-qubit gate, so its matrix is $8 \times 8$. It is the identity matrix everywhere *except* it swaps the $|110\rangle$ and $|111\rangle$ basis states. The Toffoli gate is important, combining with the Hadamard gate to form a **universal gate set** for classical reversible computation.

**2. SWAP Gate:**

* This gate swaps the states of two qubits.
* **Logic:** $|\psi_A\rangle|\psi_B\rangle \to |\psi_B\rangle|\psi_A\rangle$
* It is a $4 \times 4$ matrix which swaps the $|01\rangle$ and $|10\rangle$ basis states:

$$ 
SWAP = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}
$$

### The Tier 3 Concept: $C^n(U)$

The ultimate generalization is the **multi-controlled $U$ gate**, or $C^n(U)$.

* $n$ is the number of control qubits.
* $U$ is *any* single-qubit unitary gate (like $X$, $Z$, or a rotation $R_y(\theta)$).
* **Logic:** Apply $U$ to the target qubit *if* all $n$ control qubits are $|1\rangle$.

This gate is the key building block for complex algorithms like Shor's algorithm.

## 3. OpenQASM: The Language of Circuits

Since we can't write the matrices, we need a standard way to *describe* the circuits. The most common standard is **OpenQASM (Open Quantum Assembly Language)**. It is a simple text-based language that lists the gates in order.

**Example: The Bell State Circuit ($|\Phi^+\rangle$)**

Instead of the $4\times4$ matrix from Week 3, we write this:

```qasm
OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];  // Define a quantum register with 2 qubits, q[0] and q[1]
creg c[2];  // Define a classical register with 2 bits

h q[0];       // Apply Hadamard gate to q[0]
cx q[0], q[1];  // Apply CNOT(control=q[0], target=q[1])

measure q[0] -> c[0]; // Measure q[0] and store result in c[0]
measure q[1] -> c[1]; // Measure q[1] and store result in c[1]
```

This is how Qiskit and other quantum SDKs "think" about circuits.

## 4. The Most Important Topic: Noise

**Tier 1 Bridge:** We've been ignoring the Tier 1 concept of "static" or "decoherence." So far, all our math has assumed a perfect, ideal quantum computer. This does not exist.

**The Problem:** Qubits are fragile. Any unwanted interaction with the environment (heat, magnetic fields, etc.) can "measure" the qubit, destroying its superposition. This is called **decoherence**.

### Purpose and Formal Definition of the Density Matrix ($\rho$)

**Why do we need it?** Our `Statevector` model ($|\psi\rangle$) **cannot** handle noise.

* A state vector *only* describes **pure states** (perfectly known states, even if in superposition).
* A system that has decohered is in a **mixed state**—a classical, probabilistic "maybe" of different pure states.

Recall in Week 1, we introduced the density matrix $ρ$ to represent quantum states (pure or mixed) such that $ρ = |ψ\rangle \langle ψ|$ for pure states, and more generally $ρ = ∑ p_i |ψ_i\rangle \langle ψ_i|$ for ensembles. Its diagonals capture probabilities (populations) and off-diagonals capture coherences (superposition).
On the following section, we will see the **Density Matrix ($\rho$)** is the *true*, most general description of *any* quantum state, pure or mixed.

**Key Mathematical Tool: The Trace ($Tr$)**

Before defining $\rho$, we must define the **Trace** of a matrix, $Tr(M)$.

* **Definition:** The Trace is the sum of all the diagonal elements of a square matrix.

    $$ 
    Tr(M) = \sum_i M_{ii}
    $$ 

* **Example:**

    $$ 
    M = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix} \quad \implies \quad Tr(M) = 1 + 4 = 5
    $$ 

* **Cyclic Property:** The trace is "cyclic," which is a crucial property. For example, consider 3 different combinations of 3 different matrix $(A, B, C)$: $Tr(ABC) = Tr(CAB) = Tr(BCA)$. The trace of their product is the same for all 3 combinations.

**Formal Definition of the Density Matrix**

**Pure State:**
For a single, known pure state $|\psi\rangle$, the density matrix is defined as the "outer product":

$$ 
\rho = |\psi\rangle\langle\psi| 
$$ 

The **trace** of a pure state's density matrix is always 1. This is the new normalization condition. 

**Proof:** $Tr(\rho) = Tr(|\psi\rangle\langle\psi|)$. Using the cyclic property, we can "move" $\langle\psi|$ to the front: $Tr(\langle\psi|\psi\rangle)$. Since $\langle\psi|\psi\rangle$ is just a number (the inner product, which is 1 for a normalized state), the trace is $Tr(1) = 1$.


**Mixed State (Ensemble):**
This is the *real* power of the density matrix. If a system is in a "mixed state"—a statistical ensemble of *different* pure states—the density matrix is a weighted sum. If a system has a $p_1$ probability of being in state $|\psi_1\rangle$, a $p_2$ probability of being in state $|\psi_2\rangle$, etc., the density matrix is:

$$ 
\rho = \sum_i p_i |\psi_i\rangle\langle\psi_i| 
$$ 

**Connection to Trace:** The trace of a mixed state's density matrix is also 1, because the probabilities must sum to 1.

$$ 
Tr(\rho) = Tr(\sum_i p_i |\psi_i\rangle\langle\psi_i|) = \sum_i p_i Tr(|\psi_i\rangle\langle\psi_i|) 
$$ 

Since $Tr(|\psi_i\rangle\langle\psi_i|) = 1$ (from the pure state proof), we get: $Tr(\rho) = \sum_i p_i = 1$

In summary, the density matrix $\rho$ describes a quantum state, and the condition $Tr(\rho) = 1$ is the fundamental normalization rule, replacing $|\alpha|^2 + |\beta|^2 = 1$.

**Examples: Pure vs. Mixed States**

**1. Pure State Examples:**

* **Example:** For a qubit state $|0\rangle = \begin{pmatrix} 1 \\ 0 \end{pmatrix}$:

    $$ 
    \rho_{|0\rangle} = |0\rangle\langle 0| = \begin{pmatrix} 1 \\ 0 \end{pmatrix} \begin{pmatrix} 1 & 0 \end{pmatrix} = \begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix}
    $$ 

    *Check Trace:* $Tr(\rho_{|0\rangle}) = 1 + 0 = 1$. (Normalized)
    *The diagonal entries are the *probabilities* of measuring each basis state.*

* **Example:** For $|\psi\rangle = |+\rangle = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 \\ 1 \end{pmatrix}$:

    $$ 
    \rho_{|+\rangle} = |+\rangle\langle +| = \frac{1}{2}\begin{pmatrix} 1 \\ 1 \end{pmatrix} \begin{pmatrix} 1 & 1 \end{pmatrix} = \begin{pmatrix} 0.5 & 0.5 \\ 0.5 & 0.5 \end{pmatrix}
    $$ 

    *Check Trace:* $Tr(\rho_{|+\rangle}) = 0.5 + 0.5 = 1$. (Normalized)
    *The off-diagonal entries, "coherences," represent the superposition.*

**2. Mixed State Example:**

* **Analogy:** A "pure" $|+\rangle$ state is one coin spinning in the air. A "mixed" state is a closed box where there is a 50% chance someone put a "heads" coin ($|0\rangle$) inside and a 50% chance they put a "tails" coin ($|1\rangle$).
* **Calculation:** This is an ensemble with $p_1=0.5$ for $|0\rangle$ and $p_2=0.5$ for $|1\rangle$.

    $$ 
    \rho_{mixed} = (0.5) \cdot \rho_{|0\rangle} + (0.5) \cdot \rho_{|1\rangle} 
    $$ 

    $$ 
    \rho_{mixed} = 0.5\begin{pmatrix} 1 & 0 \\ 0 & 0 \end{pmatrix} + 0.5\begin{pmatrix} 0 & 0 \\ 0 & 1 \end{pmatrix} = \begin{pmatrix} 0.5 & 0 \\ 0 & 0.5 \end{pmatrix}
    $$ 

    *Check Trace:* $Tr(\rho_{mixed}) = 0.5 + 0.5 = 1$. (Normalized)

* This is the **maximally mixed state**. Notice the "coherences" (off-diagonals) are gone. This is what decoherence does. This looks exactly like classical randomness, which is fatal to quantum computers which rely on coherence to operate.

## 5. Formal Noise Modeling: Quantum Channels

### 5.A. The Intention & Derivation of Quantum Channels

Our ideal model of quantum evolution is unitary: $\rho_{final} = U \rho_{initial} U^\dagger$, where this means that the state or operator is in a quantum system that is closed (i.e., isolated, no interaction with an external environment).
Thus, this model *cannot* describe noise.
* **Proof:** A pure state ($\rho_{initial} = |\psi\rangle\langle\psi|$) always evolves into another pure state ($|U\psi\rangle\langle U\psi|$). It can *never* evolve into a mixed state (like the $\rho_{mixed}$ above).
* **Reality:** Noise *is* the process of a pure state "decohering" into a mixed state.
* **Intention:** We need a new mathematical object, a "super-operator" $\mathcal{E}$, that describes the evolution of a density matrix in an **open system** (a system interacting with its environment). We want to find a form for $\rho_{final} = \mathcal{E}(\rho_{initial})$.

**The Derivation:**
The derivation comes from modeling the *full* system (our "System" + "Environment") as one large, *closed* system.

1.  **Model:** We have our system (S) and its environment (E). The total system (SE) is closed, so its joint evolution *is* unitary, governed by some large $U_{SE}$.
2.  **Initial State:** We assume our system starts in state $\rho_S$ and the environment starts in a known state (e.g., $|0\rangle_E$). The total initial state is $\rho_{total} = \rho_S \otimes |0\rangle_E\langle 0|_E$.
3.  **Evolution:** The total system evolves: $\rho'_{total} = U_{SE} (\rho_S \otimes |0\rangle_E\langle 0|_E) U_{SE}^\dagger$.
4.  **Final State:** We only care about our system. We "trace out" the environment to find our system's final state, $\rho'_S$.

    $$ 
    \rho'_S = Tr_E(\rho'_{total}) = Tr_E \left( U_{SE} (\rho_S \otimes |0\rangle_E\langle 0|_E) U_{SE}^\dagger \right)
    $$ 

This derivation (which involves advanced linear algebra) simplifies to a form called the **Operator-Sum Representation (OSR)**, or **Kraus Representation**.

### 5.B. The Operator-Sum Representation (Kraus Operators)

The "Quantum Channel" $\mathcal{E}$ is formally defined by its OSR:

$$ 
\rho_{final} = \mathcal{E}(\rho_{initial}) = \sum_k E_k \rho_{initial} E_k^\dagger
$$ 

* **$\mathcal{E}$** is the **Quantum Channel** (or "super-operator").
* The $E_k$ are the **Kraus Operators**. These are operators that act *only* on our system's (S) state space.
* **Interpretation:** Each $E_k$ represents one "path" the noise can take. For example, $E_0$ could be the "nothing happens" path, $E_1$ the "bit flip" path, etc. The final state is a sum over all possible noise paths.
* **Normalization:** To conserve probability ($Tr(\rho_{final})=1$), the Kraus operators must obey $\sum_k E_k^\dagger E_k = I$.

This formalism is the "engine" of noise modeling. Any physical noise process can be described by a set of Kraus operators.

### 5.C. The Mathematics of Decoherence: The Phase Damping Channel

This is the most important noise channel for understanding decoherence. Decoherence is the process of a quantum state losing its "quantum-ness" (superposition) and becoming a simple classical probability.

Mathematically, **decoherence is the decay of the off-diagonal elements (the "coherences") of the density matrix.**

Let's prove this.
* **Our State:** A pure superposition state, $|\psi\rangle = |+\rangle = \frac{1}{\sqrt{2}}(|0\rangle + |1\rangle)$.
* **The Density Matrix:** As we saw earlier, its density matrix is:

    $$ 
    \rho_{pure} = \begin{pmatrix} 0.5 & 0.5 \\ 0.5 & 0.5 \end{pmatrix}
    $$ 

    The non-zero off-diagonal elements ($0.5$) represent the phase coherence, or superposition.

* **The Noise Model (Phase Damping):** This channel models an interaction where the environment "learns" the qubit's state in the Z-basis, which randomly applies a $Z$ gate with probability $p$. This *destroys* the phase relationship between $|0\rangle$ and $|1\rangle$.

* **Kraus Operators:**
    * $E_0 = \sqrt{1-p} \cdot I$ (Probability $1-p$, nothing happens)
    * $E_1 = \sqrt{p} \cdot Z$ (Probability $p$, a $Z$ gate is applied)
    * *Check normalization:* $\sum_k E_k^\dagger E_k = (1-p)I^\dagger I + (p)Z^\dagger Z = (1-p)I + pI = I$. (It's valid).

* **Applying the Channel:** We calculate $\rho_{final} = E_0 \rho_{pure} E_0^\dagger + E_1 \rho_{pure} E_1^\dagger$

    $$ 
    \rho_{final} = (1-p) (I \rho_{pure} I^\dagger) + p (Z \rho_{pure} Z^\dagger)
    $$ 

    * **Part 1:** $(I \rho_{pure} I^\dagger) = \rho_{pure} = \begin{pmatrix} 0.5 & 0.5 \\ 0.5 & 0.5 \end{pmatrix}$
    * **Part 2:** $Z \rho_{pure} Z^\dagger = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \begin{pmatrix} 0.5 & 0.5 \\ 0.5 & 0.5 \end{pmatrix} \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} = \begin{pmatrix} 0.5 & 0.5 \\ -0.5 & -0.5 \end{pmatrix} \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} = \begin{pmatrix} 0.5 & -0.5 \\ -0.5 & 0.5 \end{pmatrix}$

* **Combine them:**

    $$ 
    \rho_{final} = (1-p)\begin{pmatrix} 0.5 & 0.5 \\ 0.5 & 0.5 \end{pmatrix} + p\begin{pmatrix} 0.5 & -0.5 \\ -0.5 & 0.5 \end{pmatrix}
    $$ 

    $$ 
    \rho_{final} = \begin{pmatrix} 0.5(1-p) + 0.5p & 0.5(1-p) - 0.5p \\ 0.5(1-p) - 0.5p & 0.5(1-p) + 0.5p \end{pmatrix}
    $$ 

    $$ 
    \rho_{final} = \begin{pmatrix} 0.5 & 0.5(1-2p) \\ 0.5(1-2p) & 0.5 \end{pmatrix}
    $$ 

* **Analysis:**
    * The **diagonal elements (probabilities)** are *unchanged* (still 50/50).
    * The **off-diagonal elements (coherences)** have *decayed* from $0.5$ to $0.5(1-2p)$.
    * If $p=0.5$ (maximum noise), the coherences become $0.5(1-1) = 0$. The state is now:
        $$ 
        \rho_{final} = \begin{pmatrix} 0.5 & 0 \\ 0 & 0.5 \end{pmatrix}
        $$ 
    This is the **maximally mixed state** from earlier. The superposition is completely gone. This is the mathematical essence of decoherence.

### 5.D. Other Examples: Bit Flip & Depolarizing Channels

**1. Example: The Bit Flip Channel**
This channel describes a process where, with probability $p$, a qubit is hit by an $X$ gate (a bit flip). This is *not* decoherence; this is an *energy relaxation* error.

* **Kraus Operators:** $E_0 = \sqrt{1-p} \cdot I$, $E_1 = \sqrt{p} \cdot X$
* **Channel Form:** $\mathcal{E}(\rho) = (1-p) \rho + p X \rho X^\dagger$

**2. Example: The Depolarizing Channel**
This is a simple noise model that mirrors the Tier 1 "static" analogy.

* **Logic:** With probability $p$, the qubit is hit by "total noise" and its state is randomized to the maximally mixed state, $\rho_{mixed} = I/2$. With probability $(1-p)$, it is unaffected.
* **Channel Form:** $\mathcal{E}(\rho_{initial}) = (1-p) \cdot \rho_{initial} + p \cdot \frac{I}{2}$

### Conclusion

In Week 5, we have made the leap to *real* quantum computing.
1.  We generalized our gates to **$C^n(U)$** to build complex logic.
2.  We started describing circuits in **OpenQASM** instead of giant matrices.
3.  We introduced the **Density Matrix ($\rho$)** as the formal tool to describe **mixed states**.
4.  We formally defined **Quantum Channels ($\mathcal{E}$)** and their **Kraus Operators ($E_k$)** and showed mathematically how a **Phase Damping Channel** causes decoherence by eliminating the off-diagonal elements of $\rho$.